<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [21]:
# Write your code here
# Write your code here
#installing selenium package
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium





'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [22]:
from selenium import webdriver #importing selenium package
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

In [24]:
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
import pandas as pd
import time

driver = webdriver.Chrome('chromedriver',options=options)
link = 'https://www.imdb.com/title/tt8178634/reviews/?ref_=tt_ql_urv'
title_array = []
review_array = []
driver.get(link)
for num in range(4):
  driver.find_element(By.CLASS_NAME,"ipl-load-more__button").click()
  time.sleep(5)
  listOfTitle = driver.find_elements(By.CLASS_NAME, "title")
  listOfReviews = driver.find_elements(By.CLASS_NAME, "text")
for ele, sub_ele in zip(listOfTitle, listOfReviews):
      title_array.append((ele.text).replace('\n',''))
      review_array.append(sub_ele.text)
df = pd.DataFrame(list(zip(title_array, review_array)), columns =['Title', 'Review'])
print("Length of data frame is {0}".format(len(df)))
df

Length of data frame is 125


,Title,Review
0,Have Never Seen Anything Quite Like This,"I have seen a lot of movies in my time, made i..."
1,Rambo Meshed with Crouching Tiger + Musical......,I bet you'd never think the mash-up the heavy-...
2,I wish I could've seen this in a theater,There is officially ZERO reason to watch Gray ...
3,Weirdly spectacular,"When I pushed play, I did not really believe t..."
4,Wow........,This was an incredible film. I never heard of ...
...,...,...
120,"LOAD, AIM, FIRE!!!",
121,VISUAL TREAT FOR A ONE-TIME-WATCH,"The movie ran well in the box office, collecte..."
122,One more Master Piece of Emotions,
123,Awesome,Amazing movie I ever seen in my life. After a ...


In [20]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


KeyboardInterrupt: 

In [25]:
df['Lower Case'] = df['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [26]:
df['Removal of Punctuation'] = df['Lower Case'].str.replace('[^\w\s]','')

C:\Users\Supraja\AppData\Local\Temp\ipykernel_21240\1083584843.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Removal of Punctuation'] = df['Lower Case'].str.replace('[^\w\s]','')


In [27]:
import re

df['Removal of Special Characters'] = df['Removal of Punctuation'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))

In [28]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
df['Stopwords Removal'] = df['Removal of Punctuation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Supraja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
from textblob import TextBlob
df['Spelling Correction'] = df['Stopwords Removal'].apply(lambda x: str(TextBlob(x).correct()))

In [34]:
import nltk
nltk.download('punkt')
df['Tokenization'] = df['Spelling Correction'].apply(lambda x: TextBlob(x).words)

from nltk.stem import PorterStemmer
s = PorterStemmer()
df['After Stemming'] = df['Tokenization'].apply(lambda x: " ".join([s.stem(w) for w in x]))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Supraja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
from textblob import Word
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')

df['After Lemmatization'] = df['After Stemming'].apply(lambda x: " ".join([Word(w).lemmatize() for w in x.split()]))
df.to_csv('http://localhost:8888/tree/reviwes.csv',index=False)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Supraja\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Supraja\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [42]:
import itertools
import collections
from nltk import ngrams
from nltk.tokenize import word_tokenize

w_sen = []
for s in df['After Lemmatization']:
    w_sen.append(word_tokenize(s))
    a = [x1 for x1 in w_sen if x1 != []]
    iterations = list(itertools.chain.from_iterable(a))

In [46]:
tri_grams = nltk.trigrams(iterations)
f_dist = nltk.FreqDist(tri_grams)
f_dist

FreqDist({('nor', 'ram', 'charon'): 5, ('indian', 'action', 'film'): 3, ('first', 'half', 'good'): 3, ('n', 'ram', 'ran'): 3, ('ram', 'ran', 'jr'): 3, ('jr', 'ram', 'charon'): 3, ('never', 'seen', 'anyth'): 2, ('evil', 'british', 'peopl'): 2, ('movi', 'ever', 'seen'): 2, ('one', 'best', 'film'): 2, ...})

In [47]:
from collections import Counter
b_grams = nltk.bigrams(iterations)
f_dist = nltk.FreqDist(b_grams)
b_grams_dict = dict(f_dist)
for w in b_grams_dict:
    print( str(w) + ':' + str(b_grams_dict[w] / iterations.count(w[0])))

('seen', 'lot'):0.045454545454545456
('lot', 'move'):0.05
('move', 'time'):0.0425531914893617
('time', 'made'):0.019230769230769232
('made', 'lot'):0.03225806451612903
('lot', 'differ'):0.05
('differ', 'style'):0.16666666666666666
('style', 'differ'):0.16666666666666666
('differ', 'geniu'):0.16666666666666666
('geniu', 'around'):0.2
('around', 'world'):0.2222222222222222
('world', 'give'):0.09090909090909091
('give', 'seen'):0.22580645161290322
('seen', 'everyth'):0.045454545454545456
('everyth', 'mainstream'):0.0625
('mainstream', 'movi'):1.0
('movi', 'imagin'):0.004608294930875576
('imagin', 'experiment'):0.3333333333333333
('experiment', 'can'):1.0
('can', 'even'):0.15789473684210525
('even', 'rememb'):0.022222222222222223
('rememb', 'last'):0.3333333333333333
('last', 'time'):0.16666666666666666
('time', 'came'):0.019230769230769232
('came', 'away'):0.14285714285714285
('away', 'movi'):0.05
('movi', 'think'):0.009216589861751152
('think', 'id'):0.06666666666666667
('id', 'never'):0

In [48]:
w_data = ''
i1 = []
i = 1
for l in df['After Lemmatization']:
  w_data = w_data + l
  value = 'Review-' + str(i)
  i1.append(value)
  i += 1

In [49]:
import nltk
nltk.download('brown')
from textblob import TextBlob
noun_phrases = []
freq = []
for l in df['After Lemmatization']:
  blob = TextBlob(l)
  for nouns in blob.noun_phrases:
    noun_phrases.append(nouns)
for w in noun_phrases:
  noun_p_freq = []
  for line in df['After Lemmatization']:
    noun_p_freq.append(line.count(w) / w_data.count(w))
  freq.append(noun_p_freq)
noun_phrases_df = pd.DataFrame(freq).T
noun_phrases_df.columns = list(noun_phrases)
noun_phrases_df.index = i1
noun_phrases_df 

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Supraja\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


,lot differ style differ geniu,everyth mainstream movi imagin experiment,err movi,wife nearli,minut mark film top ridicul,hook total,point disappoint hour behemoth end,musclebound slickedup men fight tiger check,public flog,music number,...,visual effect action scene job,viewer stay focus screen till end,sound instant gratif someth good artist valu,amaz movi,long year u,new movi anoth nasal movieindian film industri,wast money rubbish viewer,oscar nomin,director script writer movi,maker kashmir file
Review-1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Review-121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review-122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review-123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review-124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


# **Question 2: Undersand TF-IDF and Document representation**

(20 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [51]:
# Write your code here
import math
def tf_idf_value(s,w):
  si = len(s.split(" "))
  tf_value = s.count(w)/si
  idf_value = 0
  if(tf_value!=0):
    idf_value = math.log(si)/s.count(w)
  else:
    return 0;
  return tf_value*idf_value

sentences = df["Spelling Correction"].values.tolist()
tokens_list = set([j for i in sentences for j in i.split(" ")])
tf_idf = pd.DataFrame(tokens_list,columns=["token"])
count = 0
for i in sentences:
  tf_idf[str(count)] = tf_idf["token"].apply(lambda x : tf_idf_value(i,x))
  count+=1

tf_idf

C:\Users\Supraja\AppData\Local\Temp\ipykernel_21240\1073259486.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tf_idf[str(count)] = tf_idf["token"].apply(lambda x : tf_idf_value(i,x))
C:\Users\Supraja\AppData\Local\Temp\ipykernel_21240\1073259486.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tf_idf[str(count)] = tf_idf["token"].apply(lambda x : tf_idf_value(i,x))
C:\Users\Supraja\AppData\Local\Temp\ipykernel_21240\1073259486.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

,token,0,1,2,3,4,5,6,7,8,...,115,116,117,118,119,120,121,122,123,124
0,cinematic,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.012729,...,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.00000,0.0,0.000000,0.000000
1,,0.033227,0.041835,0.08065,0.057567,0.07824,0.025863,0.04915,0.058773,0.012729,...,0.0,0.081918,0.144977,0.08065,0.119007,0.0,0.02498,0.0,0.149787,0.113373
2,canvas,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.012729,...,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.00000,0.0,0.000000,0.000000
3,any,0.033227,0.000000,0.00000,0.000000,0.00000,0.025863,0.00000,0.058773,0.012729,...,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.02498,0.0,0.000000,0.000000
4,showing,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.00000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2778,mantelpiece,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.012729,...,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.00000,0.0,0.000000,0.000000
2779,ear,0.033227,0.000000,0.00000,0.000000,0.07824,0.025863,0.04915,0.000000,0.012729,...,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.00000,0.0,0.149787,0.113373
2780,label,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.00000,0.0,0.000000,0.000000
2781,horrendous,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.00000,0.0,0.000000,0.000000


In [52]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

import nltk
nltk.download('punkt')

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Supraja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Supraja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
qry_sen = 'Shang-Chi and the Legend of the Ten Rings is an Action - Fantasy movie in which we watch a new era of Marvel Studios rise with Shang-Chi,the master of Kung Fu. He has to face his past involving him with the Ten Rings organization and crate his future accordingly.Finally, I have to say that "Shang-Chi and the Legend of the Ten Rings" is an amazing movie and I strongly recommend everyone to watch it, especially the fans of Marvel.'
X_list = word_tokenize(qry_sen)
sw = stopwords.words('english') 
X_set = {w1 for w1 in X_list if not w1 in sw}

In [54]:
def compute_vec(X_set, Y_set):
  rvec = X_set.union(Y_set)
  l1 =[];l2 =[]
  for w1 in rvec: 
      if w1 in X_set: l1.append(1)
      else: l1.append(0) 
      if w1 in Y_set: l2.append(1) 
      else: l2.append(0)
  return rvec, l1, l2

def compute_cos(rvec, l1, l2):
  c1= 0
  for i1 in range(len(rvec)): 
        c1+= l1[i1]*l2[i1] 
  cos = c1 / float((sum(l1)*sum(l2))**0.5) 
  return cos

In [55]:
similar_lst = []
for sen in df['Spelling Correction']:
  Y_list = word_tokenize(sen)
  Y_set = {w1 for w1 in Y_list if not w1 in sw}
  rvector, l1, l2 = compute_vec(X_set, Y_set)
  try:
    similar = compute_cos(rvector, l1, l2)
  except ZeroDivisionError:
    similar = 'None'
  similar_lst.append(similar)
cos_dataframe = pd.DataFrame(list(zip(df['Review'],similar_lst)), columns=['Reviews','Cosine Similarity'])
cos_dataframe

,Reviews,Cosine Similarity
0,"I have seen a lot of movies in my time, made i...",0.062137
1,I bet you'd never think the mash-up the heavy-...,0.032717
2,There is officially ZERO reason to watch Gray ...,0.024239
3,"When I pushed play, I did not really believe t...",0.062137
4,This was an incredible film. I never heard of ...,0.05135
...,...,...
120,,None
121,"The movie ran well in the box office, collecte...",0.02641
122,,None
123,Amazing movie I ever seen in my life. After a ...,0.082199


# **Question 3: Create your own word embedding model**

(20 points). Use the data you collected for assignment two to build a word embedding model: 

(1) Train a 300-dimension word embedding (it can be word2vec, glove, ulmfit, bert, or others).

(2) Visualize the word embedding model you created.

Reference: https://machinelearningmastery.com/develop-word-embeddings-python-gensim/

Reference: https://jaketae.github.io/study/word2vec/

In [56]:
# Write your code here
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [57]:
from selenium import webdriver #importing selenium package
opt = webdriver.ChromeOptions()
opt.add_argument('-headless')
opt.add_argument('-no-sandbox')
opt.add_argument('-disable-dev-shm-usage')

In [58]:
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
import pandas as pd
import time

driver = webdriver.Chrome('chromedriver',options=options)
lk = 'https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv'
tit_arr = []
revw_arr = []
driver.get(lk)
for n in range(4):
  driver.find_element(By.CLASS_NAME,"ipl-load-more__button").click()
  time.sleep(5)
  listTitle = driver.find_elements(By.CLASS_NAME, "title")
  listReviews = driver.find_elements(By.CLASS_NAME, "text")
for e, s in zip(listTitle, listReviews):
      tit_arr.append((e.text).replace('\n',''))
      revw_arr.append(s.text)
df = pd.DataFrame(list(zip(tit_arr, revw_arr)), columns =['Title', 'Review'])
print("Length of data frame is {0}".format(len(df)))
df

Length of data frame is 125


,Title,Review
0,I would not call it a masterpiece as some did,
1,Not a spoon feeding of CGI fueled faux drama.,The movie affects you in a way that makes it p...
2,Is it great? I don't know.,When I heard everyone saying that this is the ...
3,Perfect in every aspect.,"Truly a masterpiece, The Best Hollywood film o..."
4,"A psychological study, rather than a superhero...",I have seen Joker yesterday at Venice an early...
...,...,...
120,Totally overhyped.,Don't fall for it this movie is totally overhy...
121,Staying power,
122,Movie of the Year!,It was pretty clear from before the movie got ...
123,"Joaquin Phoenix, take a bow",There isn't much else to say; Joker is one of ...


In [62]:
from gensim.models import Word2Vec


# Create a list of lists of words from the "Review" column
sentences = [str(review).split() for review in df["Review"]]

# Create the Word2Vec model
mod = Word2Vec(sentences, vector_size=300)

# Print the model to verify that it was created
print(mod)

Word2Vec<vocab=491, vector_size=300, alpha=0.025>


In [65]:
from gensim.models import Word2Vec


wor2 = list(mod.wv.index_to_key)
wor2

['the',
 'and',
 'a',
 'of',
 'to',
 'is',
 'that',
 'in',
 'it',
 'I',
 'this',
 'movie',
 'as',
 'for',
 'The',
 'but',
 'with',
 'was',
 'film',
 'be',
 'Joker',
 'not',
 'you',
 'are',
 'on',
 'his',
 'by',
 'so',
 'from',
 'he',
 'an',
 'Joaquin',
 'have',
 'what',
 'all',
 'This',
 "it's",
 'Phoenix',
 'It',
 'about',
 'one',
 'or',
 'into',
 'just',
 'like',
 'at',
 'character',
 'more',
 'will',
 'than',
 'if',
 'some',
 'Arthur',
 'good',
 'really',
 'has',
 'think',
 'we',
 'can',
 'get',
 'feel',
 'performance',
 'would',
 'very',
 'no',
 'people',
 'who',
 'much',
 'But',
 'see',
 'him',
 'its',
 'it.',
 'they',
 'great',
 'And',
 'how',
 'even',
 'story',
 'your',
 'because',
 'best',
 'say',
 'do',
 'been',
 'up',
 'me',
 'seen',
 'only',
 'time',
 'many',
 'also',
 "It's",
 'Todd',
 'make',
 '-',
 'my',
 'Batman',
 'movie,',
 'Phillips',
 'A',
 'there',
 'should',
 'comic',
 'know',
 'movies',
 'Joker.',
 'movie.',
 'out',
 'most',
 'way',
 'acting',
 'any',
 'could',
 '

In [87]:

# fit a 2d PCA model to the vectors
from sklearn.decomposition import PCA

X = mod[mod.wv.index_to_key]
p = PCA(n_components=2)
res = p.fit_transform(X)

TypeError: 'Word2Vec' object is not subscriptable

In [74]:
# create a scatter plot of the projection
import matplotlib.pyplot as plt
plt.figure (figsize = (20,10))
plt.scatter(res[:, 0], res[:, 1])
word = list(mod2.wv.vocab)
for i, w in enumerate(word):
	plt.annotate(w, xy=(res[i, 0], res[i, 1]))
plt.show()

NameError: name 'res' is not defined

<Figure size 2000x1000 with 0 Axes>

# **Question 4: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [2]:
# The GitHub link of your final csv file



# Link: 



